In [2]:
import polars as pl
import altair as alt


In [4]:
board_file = "../data/runtime/bench-feather.csv"
benchmark_df = pl.read_csv(board_file)

all_correct = benchmark_df.select(pl.col("correct").all()).item(0,0)

if not all_correct:
    print("ALARM !!!")

benchmark_df

benchmark,board,correct,environment,execution_time_us,init_runtime_us,iteration,load_program_us
str,str,bool,str,i64,i64,i64,i64
"""tarfind""","""adafruit-feather-nrf52840-sens…",true,"""jerryscript""",6402631,104,0,12966
"""tarfind""","""adafruit-feather-nrf52840-sens…",true,"""jerryscript""",6396781,104,1,12966
"""tarfind""","""adafruit-feather-nrf52840-sens…",true,"""jerryscript""",6400581,104,2,12966
"""tarfind""","""adafruit-feather-nrf52840-sens…",true,"""jerryscript""",6398285,103,3,12966
"""tarfind""","""adafruit-feather-nrf52840-sens…",true,"""jerryscript""",6409804,104,4,12966
…,…,…,…,…,…,…,…
"""crc_32""","""adafruit-feather-nrf52840-sens…",true,"""wamr""",4567365,418,0,909
"""crc_32""","""adafruit-feather-nrf52840-sens…",true,"""wamr""",4565128,232,1,882
"""crc_32""","""adafruit-feather-nrf52840-sens…",true,"""wamr""",4562759,209,2,882


In [5]:
# benchmark = "tarfind"
benchmark = "crc_32"

benchmark_df = benchmark_df.filter(pl.col("benchmark").eq(benchmark))
benchmark_avg_over_iteration = benchmark_df.group_by("environment").agg(
    pl.col("execution_time_us").mean(), (pl.col("init_runtime_us").mean() + pl.col("load_program_us").mean()).alias("load_time_us")
    )

execution_time = alt.Chart(benchmark_avg_over_iteration).encode(
    alt.X("environment", title="execution environment"),
    alt.Y("execution_time_us", title="execution time in µs" ),
).mark_bar()


load_time = alt.Chart(benchmark_avg_over_iteration).encode(
    alt.X("environment", title="execution environment"),
    alt.Y("load_time_us", title="load time in µs" ),
).mark_bar()

execution_time.display(scaleFactor=2)
load_time.display(scaleFactor=2)

alt.Chart(...)

alt.Chart(...)

In [7]:
native_execution_time = benchmark_avg_over_iteration.filter(pl.col("environment").eq("native")).select("execution_time_us").item(0,0)

# print("native_execution_time", native_execution_time)

benchmark_avg_over_iteration.with_columns(
    pl.col("execution_time_us").truediv(native_execution_time).alias("times native")
)

environment,execution_time_us,load_time_us,times native
str,f64,f64,f64
"""jerryscript""",7891753.6,13883.4,327.141017
"""native""",24123.4,0.0,1.0
"""wamr""",1.011616e6,2408.2,41.935051
"""micropython""",5619891.6,42535.4,232.964325
